In [22]:

!pip install vaderSentiment


In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Concatenate, Dropout
from tensorflow.keras.models import Model
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Doc2Vec
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import pandas as pd

ENRON DATASET

In [24]:
import pandas as pd
df=messages = pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [25]:
df=df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'],axis=1)

df=df.rename(columns={"v1":"Category","v2":"Text"})

df.head()

,Category,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
df = df.rename(columns={'Category': 'label', 'Text': 'message'})
y=list(df['label'])
y=list(pd.get_dummies(y,drop_first=True)['spam'])
df['label'] = y
df.head()

,label,message
0,False,"Go until jurong point, crazy.. Available only ..."
1,False,Ok lar... Joking wif u oni...
2,True,Free entry in 2 a wkly comp to win FA Cup fina...
3,False,U dun say so early hor... U c already then say...
4,False,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Concatenate, Dropout
from tensorflow.keras.models import Model
from transformers import TFRobertaModel, RobertaTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Doc2Vec
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import pandas as pd


df = pd.read_csv('spam.csv', encoding='latin-1')
df = df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'],axis=1)
df = df.rename(columns={"v1":"Category","v2":"Text"})
df = df.rename(columns={'Category': 'label', 'Text': 'message'})

texts = df['message'].tolist()
labels = df['label'].tolist()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


Feature Extraction with RoBERTa and CNN

In [ ]:
# Load pre-trained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='tf')

roberta_input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
roberta_attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
roberta_output = roberta_model(roberta_input_ids, attention_mask=roberta_attention_mask)[0]

cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(roberta_output)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Dropout(0.5)(cnn_layer)
cnn_output = tf.keras.layers.Flatten()(cnn_layer)

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Lambda
from tensorflow.keras.models import Model
from transformers import TFRobertaModel, RobertaTokenizer

# Load pre-trained RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Define input layers
roberta_input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
roberta_attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Function to get embeddings from RoBERTa
def roberta_embedding(inputs):
    input_ids, attention_mask = inputs
    output = roberta_model(input_ids=input_ids, attention_mask=attention_mask)[0]  # Shape: (batch_size, seq_len, hidden_dim)
    return output

# Wrap RoBERTa model in a Lambda layer with explicit output shape
roberta_output = Lambda(roberta_embedding, output_shape=(128, 768))([roberta_input_ids, roberta_attention_mask])

# CNN layers
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(roberta_output)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Dropout(0.5)(cnn_layer)
cnn_output = Flatten()(cnn_layer)  # Use Flatten layer

# Define final model
model = Model(inputs=[roberta_input_ids, roberta_attention_mask], outputs=cnn_output)

# Print model summary
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 128, 768)       │              0 │ input_ids[0][0],       │
│                           │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 126, 64)        │        147,520 │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 63, 64)         │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 63, 64)         │              0 │ max_pooling1d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 4032)           │              0 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 147,520 (576.25 KB)

 Trainable params: 147,520 (576.25 KB)

 Non-trainable params: 0 (0.00 B)

 Sentence-Level Analysis with Bidirectional LSTM

In [32]:
# Bidirectional LSTM for sentence-level analysis
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(roberta_output)
lstm_layer = Dropout(0.5)(lstm_layer)
lstm_output = tf.keras.layers.Flatten()(lstm_layer)

In [33]:
# Concatenate CNN and LSTM outputs
fusion_layer = Concatenate()([cnn_output, lstm_output])

# Dense layers for final classification
dense_layer = Dense(64, activation='relu')(fusion_layer)
dense_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Define the model
model = Model(inputs=[roberta_input_ids, roberta_attention_mask], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

 Latent Dirichlet Allocation (LDA) for Topic Modeling

In [34]:
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

# LDA for topic modeling
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tfidf_matrix)

LatentDirichletAllocation(random_state=42)

 Sentiment Intensity Analysis using VADER

In [35]:
# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()
sentiments = [analyzer.polarity_scores(text)['compound'] for text in texts]

In [ ]:
X_train_features = np.hstack([cnn_output.numpy(), lstm_output.numpy(), lda.transform(tfidf_matrix), np.array(sentiments).reshape(-1, 1)])
X_test_features = np.hstack([cnn_output.numpy(), lstm_output.numpy(), lda.transform(tfidf_matrix), np.array(sentiments).reshape(-1, 1)])

# Train the model
model.fit([train_encodings['input_ids'], train_encodings['attention_mask']], y_train, epochs=5, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([test_encodings['input_ids'], test_encodings['attention_mask']], y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
model.save('email_spam_classification_model.h5')